## Creation of synthetic dataset:

In [1]:
import numpy as np
import pandas as pd
import random

def generate_pcb_dataset(num_samples=1000, board_size=(100, 100), num_components_range=(5, 15)):
    data = []
    connections = []
    obstacles_data = []

    for sample_id in range(num_samples):
        num_components = random.randint(*num_components_range)
        components = []
        obstacles = []

        num_obstacles = random.randint(1, 4)
        for obs_id in range(num_obstacles):
            obs_x = random.randint(0, board_size[0] - 10)
            obs_y = random.randint(0, board_size[1] - 10)
            obs_w = random.randint(5, 15)
            obs_h = random.randint(5, 15)
            obstacles.append((obs_x, obs_y, obs_w, obs_h))
            obstacles_data.append((sample_id, obs_id, obs_x, obs_y, obs_w, obs_h))

        for comp_id in range(num_components):
            comp_type = random.choice(["Resistor", "Capacitor", "IC", "Diode", "Inductor"])
            comp_w = random.randint(5, 20)
            comp_h = random.randint(5, 20)
            power = round(random.uniform(0.1, 5.0), 2)

            while True:
                x = random.randint(0, board_size[0] - comp_w)
                y = random.randint(0, board_size[1] - comp_h)

                if not any(
                    (x < ox + ow and x + comp_w > ox and y < oy + oh and y + comp_h > oy)
                    for ox, oy, ow, oh in obstacles
                ):
                    break

            components.append((comp_id, comp_type, x, y, comp_w, comp_h, power))

        num_connections = random.randint(1, num_components - 1)
        for _ in range(num_connections):
            comp_a, comp_b = random.sample(range(num_components), 2)
            connections.append((sample_id, comp_a, comp_b))

        for comp in components:
            data.append((sample_id, *comp))

    columns = ["SampleID", "ComponentID", "ComponentType", "X", "Y", "Width", "Height", "Power"]
    df = pd.DataFrame(data, columns=columns)

    connection_df = pd.DataFrame(connections, columns=["SampleID", "ComponentA", "ComponentB"])

    obstacle_df = pd.DataFrame(obstacles_data, columns=["SampleID", "ObstacleID", "X", "Y", "Width", "Height"])

    df.to_csv("pcb_synthetic_dataset.csv", index=False)
    connection_df.to_csv("pcb_connections.csv", index=False)
    obstacle_df.to_csv("pcb_obstacles.csv", index=False)
    print("Datasets saved: pcb_synthetic_dataset.csv, pcb_connections.csv, pcb_obstacles.csv")
    return df, connection_df, obstacle_df

df, connection_df, obstacle_df = generate_pcb_dataset()
print(df.head())
print(connection_df.head())
print(obstacle_df.head())

Datasets saved: pcb_synthetic_dataset.csv, pcb_connections.csv, pcb_obstacles.csv
   SampleID  ComponentID ComponentType   X   Y  Width  Height  Power
0         0            0         Diode  67  19     19      14   4.13
1         0            1      Inductor  61  65     10       7   0.38
2         0            2         Diode  43  41     18       8   0.62
3         0            3      Resistor  10  37     11       7   0.25
4         0            4         Diode  38  52      5      18   2.19
   SampleID  ComponentA  ComponentB
0         0           5           0
1         1           3           2
2         1           1           9
3         1           0           1
4         2           4           5
   SampleID  ObstacleID   X   Y  Width  Height
0         0           0   9  51      5      14
1         0           1  10  79      9      14
2         0           2  56  26      8       9
3         1           0  21  64     12       7
4         1           1  52  34     10      13
